# Retrieval Augumented Generation (RAG) inference

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

At this point our slide deck data is ingested into Amazon OpenSearch Service Serverless collection. We are now ready to talk to our slide deck using a large multimodal model. We are using the [Anthropic’s Claude 3 Sonnet foundation model](https://aws.amazon.com/about-aws/whats-new/2024/03/anthropics-claude-3-sonnet-model-amazon-bedrock/) for this purpose.

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [1]:
import os
import io
import sys
import json
import glob
import boto3
import codecs
import base64
import logging
import requests
import botocore
import sagemaker
import jsonlines
import numpy as np
import pandas as pd
import globals as g
from PIL import Image
from pathlib import Path
from typing import List, Dict
from IPython.display import Image
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from pandas.core.series import Series
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from utils import get_cfn_outputs, get_text_embedding, find_similar_data
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Step 2. Create an OpenSearch client and SageMaker Predictor object

We create an OpenSearch client so that we can query the vector database for embeddings (slides) similar to the questions that we might want to ask of our slide deck and then we run inference using the Claude 3 Sonnet model given the slide description we retrieved from OpenSearch.

Get the name of the OpenSearch Service Serverless collection endpoint and index name from the CloudFormation stack outputs.

In [3]:
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
index_name = outputs['OpenSearchIndexName']
logger.info(f"opensearchhost={host}, index={index_name}")


[2024-07-19 02:42:22,102] p30916 {3133969832.py:5} INFO - opensearchhost=7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com, index=blog3slides-app2


In [4]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-07-19 02:42:24,715] p30916 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Step 3. Read for RAG

We now have all the pieces for RAG. Here is how we _talk to our slide deck_.

1. Convert the user question into embeddings using the Titan Text Embeddings model.

1. Find the most similar slide (image) corresponding to the the embeddings (for the user question) from the vector database (OpenSearch Serverless).

1. Now ask Claude3 to answer the user question using the retrieved image description for the most similar slide.

In [5]:
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=g.TITAN_URL)

### Ask questions
Loop through questions in the jsonl file to -
1. Embed the question
2. Do a similarity search to retrive the closest image url and image description
3. Get final response from Claude by passing question and image description
4. Append responses to a list

Save the responses

In [6]:
llm_prompt: str = Path('prompts/desc_response_prompt.txt').read_text()

In [7]:
def get_llm_response(bedrock: botocore.client, question: str, summary: str) -> str:
    prompt = llm_prompt.format(question=question, summary=summary)
    # print(prompt)
    body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
    })
        
    try:
        response = bedrock.invoke_model(
        modelId=g.CLAUDE_MODEL_ID,
        body=body
        )

        response_body = json.loads(response['body'].read().decode("utf-8"))
        llm_response = response_body['content'][0]['text'].replace('"', "'")
        
    except Exception as e:
        logger.error(f"exception while slide_text={summary[:10]}, exception={e}")
        llm_response = None

    return llm_response

In [8]:
responses_list = []
with jsonlines.open('qa.jsonl') as f:
    for line in f.iter():
        question: str = line['question']
        question_deckname: str = line['deck_name']
        question_deckurl: str = line['deck_url']
        text_embedding = get_text_embedding(bedrock, question, g.TITAN_MODEL_ID)

        k = 4
        vector_db_response: Dict = find_similar_data(os_client, text_embedding, k, index_name, question_deckname, question_deckurl)
        response_count = len(vector_db_response.get('hits', {}).get('hits'))
        if response_count >= 1:
            for i in range(k):
                deck_name = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('metadata').get('deck_name')
                deck_url = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('metadata').get('deck_url')
                slide_text = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('slide_text')
                img_url = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('image_url')

                logger.info(f"answering the question=\"{question}\" using the summary for \"{img_url}\"")
                llm_response = get_llm_response(bedrock, question, slide_text)

                if llm_response.lower() != 'no answer':
                    response = {
                        "question": question,
                        "question_deckname": question_deckname,
                        "question_deckurl": question_deckurl,
                        "response": {
                            "resp_txt": llm_response,
                            "resp_img_url": img_url,
                            "resp_deck_name": deck_name,
                            "resp_deck_url": deck_url
                        }
                    }
                    responses_list.append(response)
                    logger.info(f"appended response corresponding to {question}")
                    break

[2024-07-19 02:42:49,049] p30916 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.220s]
[2024-07-19 02:42:49,051] p30916 {utils.py:82} INFO - received response from OpenSearch
[2024-07-19 02:42:49,051] p30916 {3118152822.py:21} INFO - answering the question="In which year did Nestlé achieve higher Organic Growth, 2003 or 2004?" using the summary for "https://image.slidesharecdn.com/2012-02-20fy11roadshow-120221022442-phpapp02/95/feb-20-2012-nestl-2011-fullyear-roadshow-presentation-7-1024.jpg"
[2024-07-19 02:42:49,852] p30916 {3118152822.py:38} INFO - appended response corresponding to In which year did Nestlé achieve higher Organic Growth, 2003 or 2004?


2004


[2024-07-19 02:42:50,146] p30916 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.063s]
[2024-07-19 02:42:50,148] p30916 {utils.py:82} INFO - received response from OpenSearch
[2024-07-19 02:42:50,149] p30916 {3118152822.py:21} INFO - answering the question="How many kinds of Business Intelligence Competence Center Key Skills are there?" using the summary for "https://image.slidesharecdn.com/agiledatascience-140624145954-phpapp02/95/agile-data-science-8-1024.jpg"
[2024-07-19 02:42:51,128] p30916 {3118152822.py:21} INFO - answering the question="How many kinds of Business Intelligence Competence Center Key Skills are there?" using the summary for "https://image.slidesharecdn.com/agile-australia-when-enterprise-meets-devops-150623023618-lva1-app6891/95/when-enterprise-meets-devops-16-1024.jpg"
[2024-07-19 02:42:52,496] p30916 {3118152822.py:21} INFO - answering the question="How many kinds of Business I

Turkey €7.8m


[2024-07-19 02:43:01,234] p30916 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.039s]
[2024-07-19 02:43:01,236] p30916 {utils.py:82} INFO - received response from OpenSearch
[2024-07-19 02:43:01,238] p30916 {3118152822.py:21} INFO - answering the question="How many points are listed for TAM Notes?" using the summary for "https://image.slidesharecdn.com/a-151012171903-lva1-app6891/95/a-guide-to-indepth-investor-pitches-by-elevate-ventures-12-1024.jpg"
[2024-07-19 02:43:02,171] p30916 {3118152822.py:21} INFO - answering the question="How many points are listed for TAM Notes?" using the summary for "https://image.slidesharecdn.com/awsustainablefinal-110317140215-phpapp01/95/case-study-sustainable-mixeduse-development-in-historic-urban-areas-10-1024.jpg"
[2024-07-19 02:43:02,978] p30916 {3118152822.py:38} INFO - appended response corresponding to How many points are listed for TAM Notes?


...

3


[2024-07-19 02:46:44,686] p30916 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.043s]
[2024-07-19 02:46:44,688] p30916 {utils.py:82} INFO - received response from OpenSearch
[2024-07-19 02:46:44,689] p30916 {3118152822.py:21} INFO - answering the question="Are there more Early Adopters or Trailblazers?" using the summary for "https://image.slidesharecdn.com/bizofchangeartofcommspublic-141115203118-conversion-gate02/95/the-business-of-change-the-art-of-communication-10-1024.jpg"
[2024-07-19 02:46:45,617] p30916 {3118152822.py:38} INFO - appended response corresponding to Are there more Early Adopters or Trailblazers?
[2024-07-19 02:46:45,739] p30916 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app2/_search [status:200 request:0.032s]
[2024-07-19 02:46:45,741] p30916 {utils.py:82} INFO - received response from OpenSearch
[2024-07-19 02:46:45,743] 

Early Adopters (18%), Trailblazers (2%)


[2024-07-19 02:46:46,519] p30916 {3118152822.py:38} INFO - appended response corresponding to What is the difference in revenue per farmer p.a. between 2005 and 2006?


$1,090


For each question, save the question and response

In [9]:
fpath: str = "responses-appr2.json"
json.dump(responses_list, codecs.open(fpath, 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)
logger.info(f"saved responses for all questions in {fpath}")

[2024-07-19 02:54:53,086] p30916 {1177031128.py:6} INFO - saved responses for all questions in responses-appr2.json
